In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

In [2]:
# Criando a base declarativa
Base = declarative_base()

/tmp/ipykernel_43457/3715230047.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Criar duas classes representando tabelas no banco de dados

In [3]:
class Usuario(Base):
    __tablename__ = 'usuarios'
    id = Column(Integer, primary_key=True)
    nome = Column(String, nullable=False)
    idade = Column(Integer, nullable=False)
    pedidos = relationship("Pedido", back_populates="usuario")

class Pedido(Base):
    __tablename__ = 'pedidos'
    id = Column(Integer, primary_key=True)
    descricao = Column(String, nullable=False)
    usuario_id = Column(Integer, ForeignKey('usuarios.id'))
    usuario = relationship("Usuario", back_populates="pedidos")

### Criar um banco de dados SQLite em memória

In [8]:
engine = create_engine('sqlite:///:memory:', echo=False)
Base.metadata.create_all(engine)

### Criar uma sessão

In [9]:
Session = sessionmaker(bind=engine)
session = Session()

### Inserir usuários e pedidos no banco de dados

In [10]:
usuario1 = Usuario(nome="Alice", idade=30)
usuario2 = Usuario(nome="Bob", idade=25)

pedido1 = Pedido(descricao="Livro de SQLAlchemy", usuario=usuario1)
pedido2 = Pedido(descricao="Notebook", usuario=usuario1)
pedido3 = Pedido(descricao="Teclado mecânico", usuario=usuario2)

session.add_all([usuario1, usuario2, pedido1, pedido2, pedido3])
session.commit()

### Consultar e exibir todos os usuários e seus pedidos

In [19]:
def exibir_usuarios_e_pedidos():
    usuarios = session.query(Usuario).all()
    for usuario in usuarios:
        print(f"Usuário: {usuario.nome}, Idade: {usuario.idade}")
        for pedido in usuario.pedidos:
            print(f"  - Pedido: {pedido.descricao}")
            
exibir_usuarios_e_pedidos()

Usuário: Alice, Idade: 35
  - Pedido: Livro de SQLAlchemy
  - Pedido: Notebook


### Atualizar a idade de um usuário

In [16]:
def atualizar_idade_usuario(nome_usuario, nova_idade):
    usuario = session.query(Usuario).filter_by(nome=nome_usuario).first()
    if usuario:
        usuario.idade = nova_idade
        session.commit()
        print(f"Idade de {nome_usuario} atualizada para {nova_idade} anos.")
    else:
        print(f"Usuário {nome_usuario} não encontrado.")

atualizar_idade_usuario("Alice", 35)

Idade de Alice atualizada para 35 anos.


### Excluir um usuário e seus pedidos associados

In [17]:
def excluir_usuario(nome_usuario):
    usuario = session.query(Usuario).filter_by(nome=nome_usuario).first()
    if usuario:
        session.delete(usuario)
        session.commit()
        print(f"Usuário {nome_usuario} e seus pedidos foram removidos.")
    else:
        print(f"Usuário {nome_usuario} não encontrado.")

excluir_usuario("Bob")

Usuário Bob e seus pedidos foram removidos.


### Buscar pedidos contendo uma palavra-chave

In [18]:
def buscar_pedidos_por_palavra_chave(palavra):
    pedidos = session.query(Pedido).filter(Pedido.descricao.contains(palavra)).all()
    print(f"Pedidos contendo '{palavra}':")
    for pedido in pedidos:
        print(f" - {pedido.descricao}, feito por {pedido.usuario.nome}")

buscar_pedidos_por_palavra_chave("Notebook")

Pedidos contendo 'Notebook':
 - Notebook, feito por Alice
